In [1]:
import sys
sys.path.append('..')
from tsfm.model.encoder import TransformerEncoderForecast, TransformerEncoderPretrain
from tsfm.eval_util.data import get_gluonts_test_dataset
from tsfm.eval_util.plot import plot_single

In [ ]:
import huggingface_hub
huggingface_hub.login(token="") # replace with your token that can access to the repo
huggingface_hub.hf_hub_download(repo_id='PeacefulData/TSFM-ScalingLaws-Checkpoints', filename='encoder_30M_16B.ckpt', local_dir='./')

In [3]:
test_data, metadata = get_gluonts_test_dataset('us_births')

pretrained_model = TransformerEncoderPretrain.load_from_checkpoint('encoder_30M_16B.ckpt')
model = TransformerEncoderForecast(
    module=pretrained_model.module,
    prediction_length=metadata.prediction_length,
    context_length=1000,
    patch_size=32,
    num_samples=1000,
    target_dim=1,
    feat_dynamic_real_dim=0,
    past_feat_dynamic_real_dim=0,
)
predictor = model.create_predictor(batch_size=1)

In [4]:
forecasts = predictor.predict(test_data.input)

In [5]:
input_iter, label_iter, forecast_iter = iter(test_data.input), iter(test_data.label), iter(forecasts)

In [7]:
inp = next(input_iter)
label = next(label_iter)
forecast = next(forecast_iter)

In [ ]:
plot_single(
    inp, 
    label, 
    forecast, 
    context_length=100,
    name="pred",
    show_label=True,
)